# Implementing physics from textbook "Physik mit Python"

# Ressources:
https://www.youtube.com/watch?v=7yZ5xxdkTb8

## 4.1 Inclined Throw

In [ ]:
import math
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.optimize import minimize, Bounds
from sklearn.neural_network import MLPRegressor
from sklearn.model_selection import train_test_split

from Modules import MassPoint, PhysicsModel # import my own modules

In [ ]:
# set initial values
h         = 10.0 # height [m]
v_ab      = 5.0  # initial velocity [m/s]
alpha_deg = 25.0 # throw angle in degrees
g         = 9.81 # earth gravity constant

In [ ]:
def inclined_throw_fun(h, v_ab, alpha_deg, g):
    mp       = MassPoint(h, v_ab, alpha_deg)
    pmodel   = PhysicsModel(h, v_ab, alpha_deg, g)
    t_e      = pmodel.time_mp_hits_earth()
    t        = pmodel.make_time_points(0,t_e,1000)
    r        = pmodel.get_spacetime_vector_of_throw(time_points=t)
    dist     = r[-1,0]
    dist_inv = 1/dist
    return r, dist, dist_inv

In [ ]:
def plot_spacetime_vector(spacetime_vector):    
    fig = plt.figure()
    ax  = fig.add_subplot(1,1,1)
    ax.set_xlabel('x [m]')
    ax.set_ylabel('y [m]')
    ax.grid()
    ax.plot(spacetime_vector[:,0], spacetime_vector[:,1])
    plt.show()

In [ ]:
r, dist, dist_inv = inclined_throw_fun(h=h, v_ab=v_ab, alpha_deg=20.0, g=g)
print('distance: ', dist)
plot_spacetime_vector(spacetime_vector=r)

In [ ]:
r1, _, _ = inclined_throw_fun(h=h, v_ab=v_ab, alpha_deg= 0.0, g=g)
r2, _, _ = inclined_throw_fun(h=h, v_ab=v_ab, alpha_deg=10.0, g=g)
r3, _, _ = inclined_throw_fun(h=h, v_ab=v_ab, alpha_deg=20.0, g=g)

# plot results
fig = plt.figure()
ax  = fig.add_subplot(1,1,1)
ax.set_xlabel('x [m]')
ax.set_ylabel('y [m]')

ax.grid()
ax.plot(r1[:,0], r1[:,1])
ax.plot(r2[:,0], r2[:,1])
ax.plot(r3[:,0], r3[:,1])
ax.legend(['alpha_deg=0','alpha_deg=10','alpha_deg=20'])
plt.show()

# Optimization: Maximize the distance under variation of the angle 

In [ ]:
_, _, dist_inv = inclined_throw_fun(h=h, v_ab=v_ab, alpha_deg=alpha_deg, g=g)
print(dist_inv)

In [ ]:
def objective_fun(x):
    h         = x[0]
    v_ab      = x[1]
    alpha_deg = x[2]
    g         = x[3]
    _, _, dist_inv = inclined_throw_fun(h=h, v_ab=v_ab, alpha_deg=alpha_deg, g=g)
    return (dist_inv - 0)**2      

In [ ]:
def optimal_angle_given_height(lb=[h, v_ab, -89.0, g], ub=[h, v_ab, 89.0, g]):
    bounds    = Bounds(lb=lb, ub=ub)
    params    = [h, v_ab, alpha_deg, g]
    res       = minimize(objective_fun, 
                         params, 
                         method = 'trust-constr', 
                         bounds = bounds)
    optimal_angle = res['constr'][0][2]
    print('The optimal angle is {:0.1f} degrees.'.format(optimal_angle))
    return optimal_angle

In [ ]:
optimal_angle_given_height(lb=[h, v_ab, -89.0, g], ub=[h, v_ab, 89.0, g])

In [ ]:
optimal_angle_list = [optimal_angle_given_height(lb=[h, v_ab, -89.0, g], ub=[h, v_ab, 89.0, g]) for h in range(0, 100)]

In [ ]:
# plot results
fig = plt.figure()
ax  = fig.add_subplot(1,1,1)
ax.set_ylim(0,50)
ax.set_xlabel('height [m]')
ax.set_ylabel('optimal angle [°]')
ax.grid()
ax.plot(optimal_angle_list)
plt.show()

# Machine Learning Application

## Model the relationship between height, initial velocity, throwing angle and gravity constant as input parameters and the according distance as output parameter

### first of all simulate data for various combinations of these parameters

In [ ]:
# loop over many possible combinations i.e. sample the parameter space as densely as possible

# save results in discitionary
data_dic = {}

counter = 0
for h in range(0,100,4):
    for v_ab in range(1,25,4):
        for alpha_deg in range(-90,90,4):
            for g in range(1,25,4): # ranging from moon to jupiter
                
                # calculate distance
                _, dist, _ = inclined_throw_fun(h=h, v_ab=v_ab, alpha_deg=alpha_deg, g=g)
                
                # save result
                data_dic[str(counter)] = [h, v_ab, alpha_deg, g, dist]
                
                # increase counter
                counter += 1

In [ ]:
df = pd.DataFrame.from_dict(data_dic, orient='index', columns=['height', 'v_ab', 'alpha_deg', 'g', 'distance'])

In [ ]:
df

### use neural network to predict the distance given the input parameters

In [ ]:
# define features and targets
X = df[['height', 'v_ab', 'alpha_deg', 'g']]
y = df[['distance']].values.reshape(-1)

In [ ]:
# split data for validation
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.8, random_state=1)

In [ ]:
regr = MLPRegressor(hidden_layer_sizes = (20,10), 
                    random_state       = 1, 
                    max_iter           = 1000)

In [ ]:
regr.fit(X_train, y_train)

In [ ]:
regr.score(X_train, y_train)

In [ ]:
regr.score(X_test, y_test)

In [ ]:
y_pred = regr.predict(X_test)

In [ ]:
plt.scatter(y_test, y_pred)
plt.xlabel('true distance [m]')
plt.ylabel('predicted distance [m]')
plt.show()

In [ ]:
def prep_input_nn(x):
    # proved a list of input variables
    return np.array([x])

In [ ]:
def distance_fun_nn(x):
    # x must be in the form of x = array([[ 63,  19, -18,  10]]) --> use prep_input_nn(x)
    distance = regr.predict(x)
    return distance

In [ ]:
def inverse_distance_fun_nn(x):
    # x must be in the form of x = array([[ 63,  19, -18,  10]]) --> use prep_input_nn(x)
    distance = regr.predict(x)
    return distance**(-1)

In [ ]:
x_test = [ 63,  19, -18,  10]
x_test

In [ ]:
x = prep_input_nn(x_test)
x

In [ ]:
distance_nn = distance_fun_nn(x)
print(distance_nn)

In [ ]:
inverse_distance_nn = inverse_distance_fun_nn(x)
print(inverse_distance_nn)

In [ ]:
def objective_fun_nn(x):
    # provide asset of parameter in a list x with: x = ['height', 'v_ab', 'alpha_deg', 'g'] 
    x = prep_input_nn(x)
    inverse_distance = inverse_distance_fun_nn(x)
    return (inverse_distance - 0)**2

In [ ]:
x_test

In [ ]:
objective_fun_nn(x_test)

In [ ]:
def optimal_angle_given_height_nn(lb=[h, v_ab, -90.0, g], ub=[h, v_ab, 90.0, g]):
    bounds    = Bounds(lb=lb, ub=ub)
    params    = [h, v_ab, alpha_deg, g]
    res       = minimize(objective_fun_nn, 
                         params, 
                         method = 'trust-constr', 
                         bounds = bounds)
    optimal_angle = res['constr'][0][2]
    print('The optimal angle is {:0.1f} degrees.'.format(optimal_angle))
    return optimal_angle

In [ ]:
optimal_angle_given_height_nn(lb=[h, v_ab, -90.0, g], ub=[h, v_ab, 90.0, g])

In [ ]:
optimal_angle_list_nn = [optimal_angle_given_height_nn(lb=[h, v_ab, -90.0, g], ub=[h, v_ab, 90.0, g]) for h in range(0, 100)]

In [ ]:
# plot results
fig = plt.figure()
ax  = fig.add_subplot(1,1,1)

ax.plot(optimal_angle_list)
ax.plot(optimal_angle_list_nn)

ax.set_ylim(0,90)
ax.set_xlabel('height [m]')
ax.set_ylabel('optimal angle [°]')
ax.legend(['true optimal angle','predicted optimal angel with nn'])
ax.grid()
plt.show()

In [ ]:
# try brute force algorithm to find optimal angle using the NN since the gradient descent algo 
#probabliy doesent work since the curve is not smooth

In [ ]:
# set initial values
h         = 10.0 # height [m]
v_ab      = 5.0  # initial velocity [m/s]
alpha_deg = 25.0 # throw angle in degrees
g         = 9.81 # earth gravity constant

In [ ]:
data_dic = {}
counter = 0
for h in range(0,100,1):
    for alpha_deg in range(-90,90,1):
        # input values
        params = np.array([[h, v_ab, alpha_deg, g]])
        # calculate distance
        distance = distance_fun_nn(params)
        distance = np.round(distance[0],2)
        # save results
        data_dic[str(counter)] = [h, v_ab, alpha_deg, g, distance]
        # increase counter
        counter +=1
        
        #print('input parameters: ', params)
        #print('throw distance: ', distance)

In [ ]:
df = pd.DataFrame.from_dict(data_dic, orient='index', columns=['height', 'v_ab', 'alpha_deg', 'g', 'distance'])

In [ ]:
df.head()

In [ ]:
distance  = df[df['height'] == 0]['distance']
alpha_deg = df[df['height'] == 0]['alpha_deg']

In [ ]:
plt.plot(alpha_deg, distance)
plt.show()

In [ ]:
h = 0
index = np.argmax(df[df['height'] == h]['distance'])
alpha_deg_opt = df[df['height'] == h].iloc[index,:]['alpha_deg']

print('height: ', h)
print('the optimal angle is: ', alpha_deg_opt)

In [ ]:
for h in range(0,100):
    index = np.argmax(df[df['height'] == h]['distance'])
    alpha_deg_opt = df[df['height'] == h].iloc[index,:]['alpha_deg']

    print('height: ', h)
    print('the optimal angle is: ', alpha_deg_opt)
    

In [ ]:
#test test

In [ ]:
#

In [ ]:
# To Do
# xxxxxx
# https://docs.scipy.org/doc/scipy/reference/tutorial/optimize.html
